In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, copy, os, shutil
from tqdm.notebook import tqdm
import seaborn as sns
from importlib import reload

# our helper functions for the gridworlds
import GridWorldHelpers as gwh

In [2]:
# initializing our environments + corresponding colors
d = 8 # dimension of our 
gw0, gw1, gw2 = gwh.build_grids(d=8, baseline_penalty = -1, water_penalty = -10, end_reward = 10)
gw0_colors = gwh.make_gw_colors(gw0)
gw1_colors = gwh.make_gw_colors(gw1)
gw2_colors = gwh.make_gw_colors(gw2)

# store quick-access indices for the environment
environments = {
                0: [gw0, gw0_colors], # baseline
                1: [gw1, gw2_colors], # non-flooding
                2: [gw2, gw2_colors] # flooding
               }

# global environment parameters 
p_switch = 0.5 # flooding Markov chain parameter
p_wind_i = 0.5 # up-down wind frequency
p_wind_j = 0.5 # left-right wind frequency

# what is our starting "current environment"
ce = 1

# which environments are we flipping through?
indices = np.array([1, 2]) # the two to be flipping between, if any. If just one, make first element

In [9]:
# make sure we are refreshing our helper functions
reload(gwh)

# what are our possible actions?
action_descs = gwh.load_actions()

# function for initializing our Q matrix, assuming 3 colors
def Q(d, init_value=0.0):
    return {((i, j, c), action) : init_value for i in range(d) for j in range(d) 
            for c in range(3) for action in list(action_descs.keys())}

In [10]:
# initialize our Q matrix: {((i, j, color), (a1, a2))}
Q = Q(d, init_value=0.0)

# counter of how many episodes we've hit
finished_episodes = 0

# things we want to store: rewards, time per iteration, Qmatrices every X steps.
rewards = []

# initialize our starting environment + starting state (bottom left corner)
env = environments[ce]; 



true_state = (d-1, 0, )

In [ ]:
environment[]

In [12]:
gw0

array([[-1., -1., -1., -1., -1., -1., -1., 10.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.]])

In [ ]:
get_environment(ce, )